# Regresión lineal multiple

Analicemos el efecto conjunto del gasto en publicidad en radio, televisión y prensa en las ventas de una compañia


![Captura de Pantalla 2021-11-05 a la(s) 22.39.26.png](https://firespring.com/wp-content/uploads/2020/06/marketing-campaigns-tv-radio-print.png)

In [ ]:
# Si estás corriendo en tu maquina local, posiblemente necesites instalar las librerias que vamos a utilizar, podrias quitar el comentario a la siguiente linea
# ! pip install sklearn statsmodels pandas numpy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from numpy.random import rand
from IPython.display import HTML
from matplotlib import animation

%matplotlib notebook


*Fuente de datos: http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv*
*An Introduction to Statistical Learning
by Gareth James, Daniela Witten, Trevor Hastie and Rob Tibshirani*

In [ ]:
# Cargamos con Pandas la tabla de datos Advertising.csv
data = pd.read_csv("Advertising.csv")

# Quitamos la columna id por que es redundante
data = data.drop(['id'], axis=1)

data.columns

In [ ]:
import seaborn as sb
print(data.corr())
dataplot = sb.heatmap(data.corr(), cmap="YlGnBu", annot=True)



Identificamos nuestras variables independientes y la dependiente. 

In [ ]:
# Xs son las variables independientes
Xs = data.drop(['ventas'], axis=1)

# y es la variable dependiente
y = data['ventas'].values.reshape(-1,1)

# Calculamos la regresión
reg = LinearRegression()
reg.fit(Xs, y)

In [ ]:
Xs.values.reshape(-1,3)

In [ ]:
intercepto = reg.intercept_[0]
coeficientes = reg.coef_[0]

print(intercepto)
print(coeficientes)

In [ ]:
coef_tv = round(coeficientes[0], 3)
coef_radio = round(coeficientes[1], 3)
coef_periodico = round(coeficientes[2], 3)

print("El módelo lineal es:")
print("Y = " + str(intercepto) + " + " + str(coef_tv) + "(tv) + " + str(coef_radio) + "(radio) + " + str(coef_periodico) + "(periodico)" )

- Observe que el coeficiente para el periódico es negativo, pero también bastante pequeño. 
- ¿Es relevante para nuestro modelo? 
**Veamos calculando el estadístico F, el valor R² y el valor p para cada coeficiente.**

In [ ]:
print("R Cuadrado:")
reg.score(Xs, y)

### Evaluar la relevancia del modelo
Como es de esperar, el procedimiento aquí es muy similar al que hicimos en la regresión lineal simple.
Convertimos las variables independientes en 

In [ ]:
# Es necesario convertir las variables independientes en un arreglo de 3 columnas, en una lista de 3 listas.
X = np.column_stack((data['tv'], data['radio'], data['periodico']))
y = data['ventas']

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

- Como puede ver, el R² es mucho más alto que el de la regresión lineal ¡con un valor de 0,897!
- Además, el estadístico F es 570,3. Esto es mucho mayor que 1, y dado que nuestro conjunto de datos es bastante pequeño (solo 200 puntos de datos), demuestra que existe una fuerte relación entre el gasto publicitario y las ventas.
- Finalmente, debido a que solo tenemos tres predictores, podemos considerar su valor p para determinar si son relevantes para el modelo o no. 
- Observa que el tercer coeficiente (el del periódico) tiene un valor p grande. Por tanto, la inversión publicitaria en periódicos no es estadísticamente significativa. 
- Eliminar ese predictor reduciría ligeramente el valor R², pero podríamos hacer mejores predicciones.

# Grafiquemos el módelo con dos variables (Radio, TV)

In [ ]:

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(data['tv'], data['radio'], data['ventas'],
           marker='o',
           alpha=1)

ax.set_xlabel('tv')
ax.set_ylabel('radio')
ax.set_zlabel('ventas')


def animate(frame):
  ax.view_init(30, frame/4)
  plt.pause(.001)
  return fig

anim = animation.FuncAnimation(fig, animate, frames=600, interval=50)
HTML(anim.to_html5_video())

In [ ]:
import statsmodels.formula.api as sm

csv = data
model = sm.ols(formula='ventas ~ tv + radio', data = csv)
fit = model.fit()

fit.summary()

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111, projection='3d')

x_surf = np.arange(0, 350, 20)                # generate a mesh
y_surf = np.arange(0, 60, 4)
x_surf, y_surf = np.meshgrid(x_surf, y_surf)

exog = pd.core.frame.DataFrame({'tv': x_surf.ravel(), 'radio': y_surf.ravel()})
out = fit.predict(exog = exog)
ax.plot_surface(x_surf, y_surf,
                out.values.reshape(x_surf.shape),
                rstride=1,
                cstride=1,
                color='red',
                alpha = 0.2)

ax.scatter(csv['tv'], csv['radio'], csv['ventas'],
           c='blue',
           marker='o',
           alpha=1)

ax.set_xlabel('TV')
ax.set_ylabel('Radio')
ax.set_zlabel('Sales')

def animate(frame):
  ax.view_init(30, frame/4)
  plt.pause(.001)
  return fig

anim = animation.FuncAnimation(fig, animate, frames=600, interval=100)
HTML(anim.to_html5_video())

In [ ]:
y